In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import lightgbm as lgbm
import catboost
from catboost import CatBoostClassifier
from ast import literal_eval

import sklearn
from sklearn.metrics import mean_absolute_error, mean_squared_error, roc_auc_score
from sklearn.model_selection import KFold, TimeSeriesSplit, StratifiedKFold, GroupKFold, train_test_split

from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation
from keras import optimizers
from keras.models import load_model

import itertools

import datetime
import time
from tqdm import tqdm

import gc

In [2]:
train = pd.read_csv('./data/train.csv')

In [41]:
train.head(30)

,game_id,winner,time,player,species,event,event_contents
0,0,1,0.00,0,T,Camera,"at (145.25, 21.5078125)"
1,0,1,0.00,1,T,Camera,"at (22.75, 147.0078125)"
2,0,1,0.02,0,T,Selection,['OrbitalCommand [3080001]']
3,0,1,0.02,0,T,Ability,(1360) - TrainSCV
4,0,1,0.14,0,T,Camera,"at (142.99609375, 24.50390625)"
5,0,1,0.14,0,T,Camera,"at (142.5078125, 24.98828125)"
6,0,1,0.14,0,T,Camera,"at (139.6171875, 27.8828125)"
7,0,1,0.14,0,T,Camera,"at (138.3359375, 29.1640625)"
8,0,1,0.14,0,T,Camera,"at (136.23828125, 31.26171875)"
9,0,1,0.14,0,T,Camera,"at (135.23828125, 32.26171875)"


In [35]:
player0_first = 0
if(player0_first==0):
    print('a')

a


In [90]:
def get_first_event(data, exchange_player =False):
    game_id = data.loc[0,'game_id']
    col = ['player0_first', 'player1_first']
    frist_train = pd.DataFrame(columns = col)

    player0_first = 0
    player1_first = 0
    for i in tqdm(range(len(data))):#len(data)
        if(data.loc[i,'game_id'] == game_id):
            # event
            if(data.loc[i,'event'] == "Ability"):
                if(player0_first==0 and data.loc[i,'player'] == 0):
                    player0_first=data.loc[i,'time']
                if(player1_first==0 and data.loc[i,'player'] == 1):
                    player1_first=data.loc[i,'time']
        else:
            df = pd.Series([player0_first, player1_first], index = col)
            frist_train = frist_train.append(df, ignore_index=True)

            # 초기화
            player0_first = 0
            player1_first = 0
            game_id = data.loc[i,'game_id']

            # event
            if(data.loc[i,'event'] == "Ability"):
                if(player0_first==0 and data.loc[i,'player'] == 0):
                    player0_first=data.loc[i,'time']
                if(player1_first==0 and data.loc[i,'player'] == 1):
                    player1_first=data.loc[i,'time']

    df = pd.Series([player0_first, player1_first], index = col)
    frist_train = frist_train.append(df, ignore_index=True)
    
    if (exchange_player == True):
        frist_train2 = frist_train.copy()
        for i in range(len(frist_train2)):
            tmp = frist_train2.loc[i,'player0_first']
            frist_train2.loc[i,'player0_first'] = frist_train2.loc[i,'player1_first']
            frist_train2.loc[i,'player1_first'] = tmp
        frist_train = pd.concat([frist_train,frist_train2])
    
    return frist_train

In [91]:
first_event_train = get_first_event(train, True)

  0%|          | 58197/33578032 [00:01<18:51, 29622.61it/s]


KeyboardInterrupt: 

In [84]:
first_event_train.to_csv('./data/train/first_event_train.csv', index = False)

In [4]:
train.head(10)

,game_id,winner,time,player,species,event,event_contents
0,0,1,0.00,0,T,Camera,"at (145.25, 21.5078125)"
1,0,1,0.00,1,T,Camera,"at (22.75, 147.0078125)"
2,0,1,0.02,0,T,Selection,['OrbitalCommand [3080001]']
3,0,1,0.02,0,T,Ability,(1360) - TrainSCV
4,0,1,0.14,0,T,Camera,"at (142.99609375, 24.50390625)"
5,0,1,0.14,0,T,Camera,"at (142.5078125, 24.98828125)"
6,0,1,0.14,0,T,Camera,"at (139.6171875, 27.8828125)"
7,0,1,0.14,0,T,Camera,"at (138.3359375, 29.1640625)"
8,0,1,0.14,0,T,Camera,"at (136.23828125, 31.26171875)"
9,0,1,0.14,0,T,Camera,"at (135.23828125, 32.26171875)"


In [15]:
X = 0.3 # 30초

In [6]:
def get_frist_train(data):
    game_id = data.loc[0,'game_id']
    col = ['winner',
           'time',
           'player0_species',
           'player0_event_Ability',
           'player0_event_AddToControlGroup',
           'player0_event_Camera',
           'player0_event_ControlGroup',
           'player0_event_GetControlGroup',
           'player0_event_RightClick',
           'player0_event_Selection',
           'player0_event_SetControlGroup',
           'player0_event_X_Ability',
           'player0_event_X_AddToControlGroup',
           'player0_event_X_Camera',
           'player0_event_X_ControlGroup',
           'player0_event_X_GetControlGroup',
           'player0_event_X_RightClick',
           'player0_event_X_Selection',
           'player0_event_X_SetControlGroup',
           'player1_species',
           'player1_event_Ability',
           'player1_event_AddToControlGroup',
           'player1_event_Camera',
           'player1_event_ControlGroup',
           'player1_event_GetControlGroup',
           'player1_event_RightClick',
           'player1_event_Selection',
           'player1_event_SetControlGroup',
           'player1_event_X_Ability',
           'player1_event_X_AddToControlGroup',
           'player1_event_X_Camera',
           'player1_event_X_ControlGroup',
           'player1_event_X_GetControlGroup',
           'player1_event_X_RightClick',
           'player1_event_X_Selection',
           'player1_event_X_SetControlGroup']
    frist_train = pd.DataFrame(columns = col)

    winner = 0
    time = 0
    player0_species = 'N'
    player0_event_Ability = []
    player0_event_AddToControlGroup = []
    player0_event_Camera = []
    player0_event_ControlGroup = []
    player0_event_GetControlGroup = []
    player0_event_RightClick = []
    player0_event_Selection = []
    player0_event_SetControlGroup = []
    player0_event_X_Ability = []
    player0_event_X_AddToControlGroup = []
    player0_event_X_Camera = []
    player0_event_X_ControlGroup = []
    player0_event_X_GetControlGroup = []
    player0_event_X_RightClick = []
    player0_event_X_Selection = []
    player0_event_X_SetControlGroup = []
    player1_species = 'N'
    player1_event_Ability = []
    player1_event_AddToControlGroup = []
    player1_event_Camera = []
    player1_event_ControlGroup = []
    player1_event_GetControlGroup = []
    player1_event_RightClick = []
    player1_event_Selection = []
    player1_event_SetControlGroup = []
    player1_event_X_Ability = []
    player1_event_X_AddToControlGroup = []
    player1_event_X_Camera = []
    player1_event_X_ControlGroup = []
    player1_event_X_GetControlGroup = []
    player1_event_X_RightClick = []
    player1_event_X_Selection = []
    player1_event_X_SetControlGroup = []
    for i in tqdm(range(len(data))):#len(data)
        if(data.loc[i,'game_id'] == game_id):
            # winner
            try:
                winner = data.loc[i,'winner']
            except Exception as e:
                _

            # time
            time = data.loc[i,'time']

            # player_species
            if(data.loc[i,'player'] == 0):
                player0_species = data.loc[i,'species']
            if(data.loc[i,'player'] == 1):
                player1_species = data.loc[i,'species']
            
            if(time <= X):
                # event
                if(data.loc[i,'event'] == "Ability"):
                    if(data.loc[i,'player'] == 0):
                        player0_event_X_Ability.append(data.loc[i,'event_contents'])
                    else:
                        player1_event_X_Ability.append(data.loc[i,'event_contents'])
                if(data.loc[i,'event'] == "AddToControlGroup"):
                    if(data.loc[i,'player'] == 0):
                        player0_event_X_AddToControlGroup.append(data.loc[i,'event_contents'])
                    else:
                        player1_event_X_AddToControlGroup.append(data.loc[i,'event_contents'])
                if(data.loc[i,'event'] == "Camera"):
                    if(data.loc[i,'player'] == 0):
                        player0_event_X_Camera.append(data.loc[i,'event_contents'])
                    else:
                        player1_event_X_Camera.append(data.loc[i,'event_contents'])
                if(data.loc[i,'event'] == "ControlGroup"):
                    if(data.loc[i,'player'] == 0):
                        player0_event_X_ControlGroup.append(data.loc[i,'event_contents'])
                    else:
                        player1_event_X_ControlGroup.append(data.loc[i,'event_contents'])
                if(data.loc[i,'event'] == "GetControlGroup"):
                    if(data.loc[i,'player'] == 0):
                        player0_event_X_GetControlGroup.append(data.loc[i,'event_contents'])
                    else:
                        player1_event_X_GetControlGroup.append(data.loc[i,'event_contents'])
                if(data.loc[i,'event'] == "Right Click"):
                    if(data.loc[i,'player'] == 0):
                        player0_event_X_RightClick.append(data.loc[i,'event_contents'])
                    else:
                        player1_event_X_RightClick.append(data.loc[i,'event_contents'])
                if(data.loc[i,'event'] == "Selection"):
                    if(data.loc[i,'player'] == 0):
                        player0_event_X_Selection.append(data.loc[i,'event_contents'])
                    else:
                        player1_event_X_Selection.append(data.loc[i,'event_contents'])
                if(data.loc[i,'event'] == "SetControlGroup"):
                    if(data.loc[i,'player'] == 0):
                        player0_event_X_SetControlGroup.append(data.loc[i,'event_contents'])
                    else:
                        player1_event_X_SetControlGroup.append(data.loc[i,'event_contents'])

            # event
            if(data.loc[i,'event'] == "Ability"):
                if(data.loc[i,'player'] == 0):
                    player0_event_Ability.append(data.loc[i,'event_contents'])
                else:
                    player1_event_Ability.append(data.loc[i,'event_contents'])
            if(data.loc[i,'event'] == "AddToControlGroup"):
                if(data.loc[i,'player'] == 0):
                    player0_event_AddToControlGroup.append(data.loc[i,'event_contents'])
                else:
                    player1_event_AddToControlGroup.append(data.loc[i,'event_contents'])
            if(data.loc[i,'event'] == "Camera"):
                if(data.loc[i,'player'] == 0):
                    player0_event_Camera.append(data.loc[i,'event_contents'])
                else:
                    player1_event_Camera.append(data.loc[i,'event_contents'])
            if(data.loc[i,'event'] == "ControlGroup"):
                if(data.loc[i,'player'] == 0):
                    player0_event_ControlGroup.append(data.loc[i,'event_contents'])
                else:
                    player1_event_ControlGroup.append(data.loc[i,'event_contents'])
            if(data.loc[i,'event'] == "GetControlGroup"):
                if(data.loc[i,'player'] == 0):
                    player0_event_GetControlGroup.append(data.loc[i,'event_contents'])
                else:
                    player1_event_GetControlGroup.append(data.loc[i,'event_contents'])
            if(data.loc[i,'event'] == "Right Click"):
                if(data.loc[i,'player'] == 0):
                    player0_event_RightClick.append(data.loc[i,'event_contents'])
                else:
                    player1_event_RightClick.append(data.loc[i,'event_contents'])
            if(data.loc[i,'event'] == "Selection"):
                if(data.loc[i,'player'] == 0):
                    player0_event_Selection.append(data.loc[i,'event_contents'])
                else:
                    player1_event_Selection.append(data.loc[i,'event_contents'])
            if(data.loc[i,'event'] == "SetControlGroup"):
                if(data.loc[i,'player'] == 0):
                    player0_event_SetControlGroup.append(data.loc[i,'event_contents'])
                else:
                    player1_event_SetControlGroup.append(data.loc[i,'event_contents'])
        else:
            df = pd.Series([winner, 
                            time,
                            player0_species,
                            player0_event_Ability,
                            player0_event_AddToControlGroup,
                            player0_event_Camera,
                            player0_event_ControlGroup,
                            player0_event_GetControlGroup,
                            player0_event_RightClick,
                            player0_event_Selection,
                            player0_event_SetControlGroup,
                            player0_event_X_Ability,
                            player0_event_X_AddToControlGroup,
                            player0_event_X_Camera,
                            player0_event_X_ControlGroup,
                            player0_event_X_GetControlGroup,
                            player0_event_X_RightClick,
                            player0_event_X_Selection,
                            player0_event_X_SetControlGroup,
                            player1_species,
                            player1_event_Ability,
                            player1_event_AddToControlGroup,
                            player1_event_Camera,
                            player1_event_ControlGroup,
                            player1_event_GetControlGroup,
                            player1_event_RightClick,
                            player1_event_Selection,
                            player1_event_SetControlGroup,
                            player1_event_X_Ability,
                            player1_event_X_AddToControlGroup,
                            player1_event_X_Camera,
                            player1_event_X_ControlGroup,
                            player1_event_X_GetControlGroup,
                            player1_event_X_RightClick,
                            player1_event_X_Selection,
                            player1_event_X_SetControlGroup], index = col)
            frist_train = frist_train.append(df, ignore_index=True)

            # 초기화
            winner = 0
            time = 0
            player0_species = 'N'
            player0_event_Ability = []
            player0_event_AddToControlGroup = []
            player0_event_Camera = []
            player0_event_ControlGroup = []
            player0_event_GetControlGroup = []
            player0_event_RightClick = []
            player0_event_Selection = []
            player0_event_SetControlGroup = []
            player0_event_X_Ability = []
            player0_event_X_AddToControlGroup = []
            player0_event_X_Camera = []
            player0_event_X_ControlGroup = []
            player0_event_X_GetControlGroup = []
            player0_event_X_RightClick = []
            player0_event_X_Selection = []
            player0_event_X_SetControlGroup = []
            player1_species = 'N'
            player1_event_Ability = []
            player1_event_AddToControlGroup = []
            player1_event_Camera = []
            player1_event_ControlGroup = []
            player1_event_GetControlGroup = []
            player1_event_RightClick = []
            player1_event_Selection = []
            player1_event_SetControlGroup = []
            player1_event_X_Ability = []
            player1_event_X_AddToControlGroup = []
            player1_event_X_Camera = []
            player1_event_X_ControlGroup = []
            player1_event_X_GetControlGroup = []
            player1_event_X_RightClick = []
            player1_event_X_Selection = []
            player1_event_X_SetControlGroup = []
            
            game_id = data.loc[i,'game_id']
            # winner
            try:
                winner = data.loc[i,'winner']
            except Exception as e:
                _

            # time
            time = data.loc[i,'time']

            # player_species
            if(data.loc[i,'player'] == 0):
                player0_species = data.loc[i,'species']
            if(data.loc[i,'player'] == 1):
                player1_species = data.loc[i,'species']
            
            if(time <= X):
                # event
                if(data.loc[i,'event'] == "Ability"):
                    if(data.loc[i,'player'] == 0):
                        player0_event_X_Ability.append(data.loc[i,'event_contents'])
                    else:
                        player1_event_X_Ability.append(data.loc[i,'event_contents'])
                if(data.loc[i,'event'] == "AddToControlGroup"):
                    if(data.loc[i,'player'] == 0):
                        player0_event_X_AddToControlGroup.append(data.loc[i,'event_contents'])
                    else:
                        player1_event_X_AddToControlGroup.append(data.loc[i,'event_contents'])
                if(data.loc[i,'event'] == "Camera"):
                    if(data.loc[i,'player'] == 0):
                        player0_event_X_Camera.append(data.loc[i,'event_contents'])
                    else:
                        player1_event_X_Camera.append(data.loc[i,'event_contents'])
                if(data.loc[i,'event'] == "ControlGroup"):
                    if(data.loc[i,'player'] == 0):
                        player0_event_X_ControlGroup.append(data.loc[i,'event_contents'])
                    else:
                        player1_event_X_ControlGroup.append(data.loc[i,'event_contents'])
                if(data.loc[i,'event'] == "GetControlGroup"):
                    if(data.loc[i,'player'] == 0):
                        player0_event_X_GetControlGroup.append(data.loc[i,'event_contents'])
                    else:
                        player1_event_X_GetControlGroup.append(data.loc[i,'event_contents'])
                if(data.loc[i,'event'] == "Right Click"):
                    if(data.loc[i,'player'] == 0):
                        player0_event_X_RightClick.append(data.loc[i,'event_contents'])
                    else:
                        player1_event_X_RightClick.append(data.loc[i,'event_contents'])
                if(data.loc[i,'event'] == "Selection"):
                    if(data.loc[i,'player'] == 0):
                        player0_event_X_Selection.append(data.loc[i,'event_contents'])
                    else:
                        player1_event_X_Selection.append(data.loc[i,'event_contents'])
                if(data.loc[i,'event'] == "SetControlGroup"):
                    if(data.loc[i,'player'] == 0):
                        player0_event_X_SetControlGroup.append(data.loc[i,'event_contents'])
                    else:
                        player1_event_X_SetControlGroup.append(data.loc[i,'event_contents'])

            # event
            if(data.loc[i,'event'] == "Ability"):
                if(data.loc[i,'player'] == 0):
                    player0_event_Ability.append(data.loc[i,'event_contents'])
                else:
                    player1_event_Ability.append(data.loc[i,'event_contents'])
            if(data.loc[i,'event'] == "AddToControlGroup"):
                if(data.loc[i,'player'] == 0):
                    player0_event_AddToControlGroup.append(data.loc[i,'event_contents'])
                else:
                    player1_event_AddToControlGroup.append(data.loc[i,'event_contents'])
            if(data.loc[i,'event'] == "Camera"):
                if(data.loc[i,'player'] == 0):
                    player0_event_Camera.append(data.loc[i,'event_contents'])
                else:
                    player1_event_Camera.append(data.loc[i,'event_contents'])
            if(data.loc[i,'event'] == "ControlGroup"):
                if(data.loc[i,'player'] == 0):
                    player0_event_ControlGroup.append(data.loc[i,'event_contents'])
                else:
                    player1_event_ControlGroup.append(data.loc[i,'event_contents'])
            if(data.loc[i,'event'] == "GetControlGroup"):
                if(data.loc[i,'player'] == 0):
                    player0_event_GetControlGroup.append(data.loc[i,'event_contents'])
                else:
                    player1_event_GetControlGroup.append(data.loc[i,'event_contents'])
            if(data.loc[i,'event'] == "Right Click"):
                if(data.loc[i,'player'] == 0):
                    player0_event_RightClick.append(data.loc[i,'event_contents'])
                else:
                    player1_event_RightClick.append(data.loc[i,'event_contents'])
            if(data.loc[i,'event'] == "Selection"):
                if(data.loc[i,'player'] == 0):
                    player0_event_Selection.append(data.loc[i,'event_contents'])
                else:
                    player1_event_Selection.append(data.loc[i,'event_contents'])
            if(data.loc[i,'event'] == "SetControlGroup"):
                if(data.loc[i,'player'] == 0):
                    player0_event_SetControlGroup.append(data.loc[i,'event_contents'])
                else:
                    player1_event_SetControlGroup.append(data.loc[i,'event_contents'])

    df = pd.Series([winner, 
                    time,
                    player0_species,
                    player0_event_Ability,
                    player0_event_AddToControlGroup,
                    player0_event_Camera,
                    player0_event_ControlGroup,
                    player0_event_GetControlGroup,
                    player0_event_RightClick,
                    player0_event_Selection,
                    player0_event_SetControlGroup,
                    player0_event_X_Ability,
                    player0_event_X_AddToControlGroup,
                    player0_event_X_Camera,
                    player0_event_X_ControlGroup,
                    player0_event_X_GetControlGroup,
                    player0_event_X_RightClick,
                    player0_event_X_Selection,
                    player0_event_X_SetControlGroup,
                    player1_species,
                    player1_event_Ability,
                    player1_event_AddToControlGroup,
                    player1_event_Camera,
                    player1_event_ControlGroup,
                    player1_event_GetControlGroup,
                    player1_event_RightClick,
                    player1_event_Selection,
                    player1_event_SetControlGroup,
                    player1_event_X_Ability,
                    player1_event_X_AddToControlGroup,
                    player1_event_X_Camera,
                    player1_event_X_ControlGroup,
                    player1_event_X_GetControlGroup,
                    player1_event_X_RightClick,
                    player1_event_X_Selection,
                    player1_event_X_SetControlGroup], index = col)
    frist_train = frist_train.append(df, ignore_index=True)
    return frist_train

In [7]:
frist_train = get_frist_train(train)

100%|██████████| 1000000/1000000 [04:23<00:00, 3796.42it/s]


In [8]:
frist_train.head(10)

,winner,time,player0_species,player0_event_Ability,player0_event_AddToControlGroup,player0_event_Camera,player0_event_ControlGroup,player0_event_GetControlGroup,player0_event_RightClick,player0_event_Selection,...,player1_event_Selection,player1_event_SetControlGroup,player1_event_X_Ability,player1_event_X_AddToControlGroup,player1_event_X_Camera,player1_event_X_ControlGroup,player1_event_X_GetControlGroup,player1_event_X_RightClick,player1_event_X_Selection,player1_event_X_SetControlGroup
0,1,7.24,T,"[(1360) - TrainSCV, (1021) - BuildSupplyDepot;...","[nan, nan]","[at (145.25, 21.5078125), at (142.99609375, 24...",[],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[Location: (136.3759765625, 41.11962890625, 49...","[['OrbitalCommand [3080001]'], ['OrbitalComman...",...,"[['OrbitalCommand [33C0001]'], ['SCV [3400001]...",[nan],[],[],"[at (22.75, 147.0078125)]",[],[],[],[],[]
1,0,7.28,Z,"[(1820) - MorphDrone, (4C1) - Patrol; Location...",[],"[at (24.75, 128.0078125), at (24.75, 125.99218...",[],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[Target: None [00700001]; Location: (19.0, 129...","[['Hive [3100001]'], ['Egg [3140001]', 'Egg [3...",...,"[['Nexus [3540001]'], ['Probe [3580001]', 'Pro...","[nan, nan, nan, nan, nan]","[(15E0) - TrainProbe, (1541) - BuildPylon; Loc...",[],"[at (151.25, 20.5078125), at (148.45703125, 49...",[],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[Location: (150.21923828125, 19.0400390625, 49...","[['Nexus [3540001]'], ['Probe [3580001]', 'Pro...","[nan, nan]"
2,1,4.27,P,"[(15E0) - TrainProbe, (15E0) - TrainProbe, (15...",[nan],"[at (140.0, 162.0078125), at (138.5, 160.75781...",[],"[nan, nan, nan]",[Target: DestructibleSignsConstruction [003800...,"[['Probe [3200001]'], ['Probe [3180001]'], ['N...",...,"[['Nexus [3340001]'], ['Nexus [3340001]'], ['P...","[nan, nan, nan, nan, nan]","[(15E0) - TrainProbe, (15E0) - TrainProbe, (15...",[],"[at (28.0, 18.5078125), at (29.5, 19.7578125),...",[],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[Location: (30.15380859375, 16.108642578125, 4...","[['Nexus [3340001]'], ['Nexus [3340001]'], ['P...",[nan]
3,0,9.59,T,"[(1360) - TrainSCV, (1360) - TrainSCV, (1021) ...","[nan, nan, nan, nan, nan, nan]","[at (145.25, 21.53515625), at (145.25, 22.7382...",[],"[nan, nan, nan, nan, nan, nan, nan, nan]","[Location: (135.513671875, 39.0166015625, 4912...","[['SCV [3740001]'], ['OrbitalCommand [3080001]...",...,"[['Nexus [33C0001]'], ['Probe [3540001]'], ['P...",[nan],"[(15E0) - TrainProbe, (15E0) - TrainProbe, (15...",[nan],"[at (22.75, 147.0078125), at (21.3203125, 116....",[],[nan],"[Target: None [01680001]; Location: (18.0, 149...","[['Nexus [33C0001]'], ['Probe [3540001]']]",[]
4,1,4.55,Z,"[(1820) - MorphDrone, (16E0) - BuildHatchery; ...",[],"[at (127.25, 27.5078125), at (125.9375, 28.820...",[],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[Location: (116.93994140625, 46.2587890625, 47...","[['Hatchery [3380001]'], ['Egg [33C0001]', 'Eg...",...,"[['Hatchery [37C0001]'], ['Egg [3800001]', 'Eg...","[nan, nan, nan, nan, nan, nan, nan]","[(1820) - MorphDrone, (1822) - MorphOverlord]",[],"[at (24.75, 137.0078125), at (119.15234375, 61...",[],"[nan, nan, nan, nan, nan]",[Target: DestructibleSignsConstruction [026000...,"[['Hatchery [37C0001]'], ['Egg [3800001]', 'Eg...","[nan, nan]"
5,0,10.18,Z,"[(1820) - MorphDrone, (1822) - MorphOverlord, ...","[nan, nan]","[at (151.25, 20.5078125), at (151.25, 22.17187...",[],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[Location: (138.77685546875, 26.192138671875, ...","[['Egg [3140001]', 'Egg [3180001]', 'Egg [31C0...",...,"[['DestructibleSignsConstruction [740001]'], [...","[nan, nan, nan, nan, nan, nan, nan, nan, nan]","[(1360) - TrainSCV, (1021) - BuildSupplyDepot;...",[],"[at (24.75, 128.0078125), at (22.05859375, 125...",[],[],[Target: DestructibleSignsConstruction [007400...,"[['DestructibleSignsConstruction [740001]'], [...",[]
6,1,10.36,Z,"[(1820) - MorphDrone, (16E2) - BuildExtractor

In [9]:
# 데이터 저장
frist_train.to_csv('./data/train/first_X_train.csv',index=False)

In [2]:
# 데이터 로드
frist_train = pd.read_csv('./data/train/first_X_train.csv')

In [4]:
frist_train

,winner,time,player0_species,player0_event_Ability,player0_event_AddToControlGroup,player0_event_Camera,player0_event_ControlGroup,player0_event_GetControlGroup,player0_event_RightClick,player0_event_Selection,...,player1_event_Selection,player1_event_SetControlGroup,player1_event_X_Ability,player1_event_X_AddToControlGroup,player1_event_X_Camera,player1_event_X_ControlGroup,player1_event_X_GetControlGroup,player1_event_X_RightClick,player1_event_X_Selection,player1_event_X_SetControlGroup
0,1,7.24,T,"['(1360) - TrainSCV', '(1021) - BuildSupplyDep...","[nan, nan]","['at (145.25, 21.5078125)', 'at (142.99609375,...",[],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","['Location: (136.3759765625, 41.11962890625, 4...","[""['OrbitalCommand [3080001]']"", ""['OrbitalCom...",...,"[""['OrbitalCommand [33C0001]']"", ""['SCV [34000...",[nan],[],[],"['at (22.75, 147.0078125)']",[],[],[],[],[]
1,0,7.28,Z,"['(1820) - MorphDrone', '(4C1) - Patrol; Locat...",[],"['at (24.75, 128.0078125)', 'at (24.75, 125.99...",[],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","['Target: None [00700001]; Location: (19.0, 12...","[""['Hive [3100001]']"", ""['Egg [3140001]', 'Egg...",...,"[""['Nexus [3540001]']"", ""['Probe [3580001]', '...","[nan, nan, nan, nan, nan]","['(15E0) - TrainProbe', '(1541) - BuildPylon; ...",[],"['at (151.25, 20.5078125)', 'at (148.45703125,...",[],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","['Location: (150.21923828125, 19.0400390625, 4...","[""['Nexus [3540001]']"", ""['Probe [3580001]', '...","[nan, nan]"
2,1,4.27,P,"['(15E0) - TrainProbe', '(15E0) - TrainProbe',...",[nan],"['at (140.0, 162.0078125)', 'at (138.5, 160.75...",[],"[nan, nan, nan]",['Target: DestructibleSignsConstruction [00380...,"[""['Probe [3200001]']"", ""['Probe [3180001]']"",...",...,"[""['Nexus [3340001]']"", ""['Nexus [3340001]']"",...","[nan, nan, nan, nan, nan]","['(15E0) - TrainProbe', '(15E0) - TrainProbe',...",[],"['at (28.0, 18.5078125)', 'at (29.5, 19.757812...",[],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","['Location: (30.15380859375, 16.108642578125, ...","[""['Nexus [3340001]']"", ""['Nexus [3340001]']"",...",[nan]
3,0,9.59,T,"['(1360) - TrainSCV', '(1360) - TrainSCV', '(1...","[nan, nan, nan, nan, nan, nan]","['at (145.25, 21.53515625)', 'at (145.25, 22.7...",[],"[nan, nan, nan, nan, nan, nan, nan, nan]","['Location: (135.513671875, 39.0166015625, 491...","[""['SCV [3740001]']"", ""['OrbitalCommand [30800...",...,"[""['Nexus [33C0001]']"", ""['Probe [3540001]']"",...",[nan],"['(15E0) - TrainProbe', '(15E0) - TrainProbe',...",[nan],"['at (22.75, 147.0078125)', 'at (21.3203125, 1...",[],[nan],"['Target: None [01680001]; Location: (18.0, 14...","[""['Nexus [33C0001]']"", ""['Probe [3540001]']""]",[]
4,1,4.55,Z,"['(1820) - MorphDrone', '(16E0) - BuildHatcher...",[],"['at (127.25, 27.5078125)', 'at (125.9375, 28....",[],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","['Location: (116.93994140625, 46.2587890625, 4...","[""['Hatchery [3380001]']"", ""['Egg [33C0001]', ...",...,"[""['Hatchery [37C0001]']"", ""['Egg [3800001]', ...","[nan, nan, nan, nan, nan, nan, nan]","['(1820) - MorphDrone', '(1822) - MorphOverlord']",[],"['at (24.75, 137.0078125)', 'at (119.15234375,...",[],"[nan, nan, nan, nan, nan]",['Target: DestructibleSignsConstruction [02600...,"[""['Hatchery [37C0001]']"", ""['Egg [3800001]', ...","[nan, nan]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19431,1,0.39,P,"['(15E0) - TrainProbe', '(15E0) - TrainProbe',...",[],"['at (28.0, 18.5078125)', 'at (28.28125, 19.35...",[],[nan],"['Location: (88.50341796875, 142.006591796875,...","[""['Probe [3040001]']""]",...,"[""['Overlord [3740001]']"", ""['Drone [3500001]'...",[],['(16E3) - BuildSpawningPool; Location: (131.5...,[],"['at (140.0, 162.0078125)', 'at (140.0, 161.41...",[],[],"['Location: (44.63134765625, 20.3720703125, 49...","[""['Overlord [3740001]']"", ""['Drone [3500001]'...",[]
19432,1,0.37,T,"['(1360) - Tr

In [3]:
def get_double_train(data):
    data_copy = data.copy()
    for i in tqdm(range(len(data))):#len(data)
        for j in range(len(data_copy.columns)//2+1):
            if(j==0):
                if(data_copy.iloc[i,j] == 0):
                    data_copy.iloc[i,j] = 1
                else:
                    data_copy.iloc[i,j] = 0

            elif(j==1):
                _
            else:
                tmp = data_copy.iloc[i,j]
                data_copy.iloc[i,j] = data_copy.iloc[i,j+17]
                data_copy.iloc[i,j+17] = tmp
    return data_copy

In [326]:
frist_train_copy = get_double_train(frist_train)

100%|██████████| 19436/19436 [07:26<00:00, 43.49it/s]


In [327]:
double_train = pd.concat([frist_train, frist_train_copy])

In [328]:
# 데이터 저장
double_train.to_csv('./data/train/double_train.csv',index=False)

In [329]:
# 데이터 로드
double_train = pd.read_csv('./data/train/double_train.csv')

In [330]:
def get_units_train(data):
    data = data.assign(
    player0_SCV= 0,
    player0_Mule = 0,
    player0_Marine = 0 ,
    player0_Marauder = 0 ,
    player0_Reaper = 0 ,
    player0_Ghost = 0 ,
    player0_Hellion = 0 ,
    player0_SiegeTank= 0 , 
    player0_Thor= 0,
    player0_Viking = 0,
    player0_Medivac = 0,
    player0_Banshee = 0,
    player0_Raven = 0,
    player0_AutoTurret = 0,
    player0_Battlecruiser = 0,
    player0_Probe=0,
    player0_MothershipCore = 0 ,
    player0_Mothership=0,
    player0_Zealot=0,
    player0_Stalker=0,
    player0_Sentry=0,
    player0_HighTemplar=0,
    player0_DarkTemplar=0,
    player0_Archon=0,
    player0_Immortal=0,
    player0_Observer=0,
    player0_WarpPrism=0,
    player0_Colossus=0,
    player0_Disruptor=0,
    player0_Phoenix=0,
    player0_Carrier=0,
    player0_VoidRay=0,
    player0_Tempest=0,
    player0_Oracle=0,
    player0_Larva = 0,
    player0_Drone = 0,
    player0_Overlord = 0,
    player0_Overseer = 0,
    player0_changeling = 0,
    player0_Zergling = 0,
    player0_Baneling = 0,
    player0_Queen = 0,
    player0_Hydralisk = 0,
    player0_Roach = 0,
    player0_Mutalisk = 0,
    player0_Corruptor = 0,
    player0_BroodLord = 0,
    player0_Broodling = 0,
    player0_Infestor = 0,
    player0_InfestedTerran = 0,
    player0_Ultralisk = 0,
    player1_SCV= 0,
    player1_Mule = 0,
    player1_Marine = 0 ,
    player1_Marauder = 0 ,
    player1_Reaper = 0 ,
    player1_Ghost = 0 ,
    player1_Hellion = 0 ,
    player1_SiegeTank= 0 , 
    player1_Thor= 0,
    player1_Viking = 0,
    player1_Medivac = 0,
    player1_Banshee = 0,
    player1_Raven = 0,
    player1_AutoTurret = 0,
    player1_Battlecruiser = 0,
    player1_Probe=0,
    player1_MothershipCore = 0 ,
    player1_Mothership=0,
    player1_Zealot=0,
    player1_Stalker=0,
    player1_Sentry=0,
    player1_HighTemplar=0,
    player1_DarkTemplar=0,
    player1_Archon=0,
    player1_Immortal=0,
    player1_Observer=0,
    player1_WarpPrism=0,
    player1_Colossus=0,
    player1_Disruptor=0,
    player1_Phoenix=0,
    player1_Carrier=0,
    player1_VoidRay=0,
    player1_Tempest=0,
    player1_Oracle=0,
    player1_Larva = 0,
    player1_Drone = 0,
    player1_Overlord = 0,
    player1_Overseer = 0,
    player1_changeling = 0,
    player1_Zergling = 0,
    player1_Baneling = 0,
    player1_Queen = 0,
    player1_Hydralisk = 0,
    player1_Roach = 0,
    player1_Mutalisk = 0,
    player1_Corruptor = 0,
    player1_BroodLord = 0,
    player1_Broodling = 0,
    player1_Infestor = 0,
    player1_InfestedTerran = 0,
    player1_Ultralisk = 0)

    for i in tqdm(range(len(data))):
        e=data['player0_event_Ability'][i]
        if data['player0_species'][i] == 'T':
            player0_SCV =e.count('SCV')
            data['player0_SCV'][i] = player0_SCV
            player0_Mule =e.count('MULE')
            data['player0_Mule'][i] = player0_Mule
            player0_Marine =e.count('Marine')
            data['player0_Marine'][i] = player0_Marine
            player0_Marauder =e.count('Marauder')
            data['player0_Marauder'][i] = player0_Marauder
            player0_Reaper =e.count('Reaper')
            data['player0_Reaper'][i] = player0_Reaper
            player0_Ghost =e.count('Ghost')
            data['player0_Ghost'][i] = player0_Ghost
            player0_Hellion =e.count('Hellion')
            data['player0_Hellion'][i] = player0_Hellion
            player0_SiegeTank =e.count('SiegeTank')
            data['player0_SiegeTank'][i] = player0_SiegeTank
            data['player0_Thor'][i] = e.count('Thor')
            data['player0_Viking'][i] = e.count('Viking')
            data['player0_Medivac'][i] = e.count('Medivac')
            data['player0_Banshee'][i] = e.count('Banshee')
            data['player0_Raven'][i] = e.count('Raven')
            data['player0_AutoTurret'][i] = e.count('AutoTurret')
            data['player0_Battlecruiser'][i] = e.count('Battlecruiser')
        elif data['player0_species'][i] == 'P':
            data['player0_Probe'][i] = e.count('Probe')
            data['player0_MothershipCore'][i] = e.count('MothershipCore')
            data['player0_Mothership'][i] = e.count('Mothership')
            data['player0_Zealot'][i] = e.count('Zealot')
            data['player0_Stalker'][i] = e.count('Stalker')
            data['player0_Sentry'][i] = e.count('Sentry')
            data['player0_HighTemplar'][i] = e.count('HighTemplar')
            data['player0_DarkTemplar'][i] = e.count('DarkTemplar')
            data['player0_Archon'][i] = e.count('Archon')
            data['player0_Immortal'][i] = e.count('Immortal')
            data['player0_Observer'][i] = e.count('Observer')
            data['player0_WarpPrism'][i] = e.count('WarpPrism')
            data['player0_Colossus'][i] = e.count('Colossus')
            data['player0_Disruptor'][i] = e.count('Disruptor')
            data['player0_Phoenix'][i] = e.count('Phoenix')
            data['player0_Carrier'][i] = e.count('Carrier')
            data['player0_VoidRay'][i] = e.count('VoidRay')
            data['player0_Tempest'][i] = e.count('Tempest')
            data['player0_Oracle'][i] = e.count('Oracle')
        elif data['player0_species'][i] == 'Z':
            data['player0_Larva'][i] = e.count('Larva')
            data['player0_Drone'][i] = e.count('Drone')
            data['player0_Overlord'][i] = e.count('Overlord')
            data['player0_Overseer'][i] = e.count('Overseer')
            data['player0_changeling'][i] = e.count('changeling')
            data['player0_Zergling'][i] = e.count('Zergling')
            data['player0_Baneling'][i] = e.count('Baneling')
            data['player0_Queen'][i] = e.count('Queen')
            data['player0_Hydralisk'][i] = e.count('Hydralisk')
            data['player0_Roach'][i] = e.count('Roach')
            data['player0_Mutalisk'][i] = e.count('Mutalisk')
            data['player0_Corruptor'][i] = e.count('Corruptor')
            data['player0_BroodLord'][i] = e.count('BroodLord')
            data['player0_Broodling'][i] = e.count('Broodling')
            data['player0_Infestor'][i] = e.count('Infestor')
            data['player0_InfestedTerran'][i] = e.count('InfestedTerran')
            data['player0_Ultralisk'][i] = e.count('Ultralisk')

        e=data['player1_event_Ability'][i]
        if data['player1_species'][i] == 'T':
            player1_SCV =e.count('SCV')
            data['player1_SCV'][i] = player1_SCV
            player1_Mule =e.count('MULE')
            data['player1_Mule'][i] = player1_Mule
            player1_Marine =e.count('Marine')
            data['player1_Marine'][i] = player1_Marine
            player1_Marauder =e.count('Marauder')
            data['player1_Marauder'][i] = player1_Marauder
            player1_Reaper =e.count('Reaper')
            data['player1_Reaper'][i] = player1_Reaper
            player1_Ghost =e.count('Ghost')
            data['player1_Ghost'][i] = player1_Ghost
            player1_Hellion =e.count('Hellion')
            data['player1_Hellion'][i] = player1_Hellion
            player1_SiegeTank =e.count('SiegeTank')
            data['player1_SiegeTank'][i] = player1_SiegeTank
            data['player1_Thor'][i] = e.count('Thor')
            data['player1_Viking'][i] = e.count('Viking')
            data['player1_Medivac'][i] = e.count('Medivac')
            data['player1_Banshee'][i] = e.count('Banshee')
            data['player1_Raven'][i] = e.count('Raven')
            data['player1_AutoTurret'][i] = e.count('AutoTurret')
            data['player1_Battlecruiser'][i] = e.count('Battlecruiser')
        elif data['player1_species'][i] == 'P':
            data['player1_Probe'][i] = e.count('Probe')
            data['player1_MothershipCore'][i] = e.count('MothershipCore')
            data['player1_Mothership'][i] = e.count('Mothership')
            data['player1_Zealot'][i] = e.count('Zealot')
            data['player1_Stalker'][i] = e.count('Stalker')
            data['player1_Sentry'][i] = e.count('Sentry')
            data['player1_HighTemplar'][i] = e.count('HighTemplar')
            data['player1_DarkTemplar'][i] = e.count('DarkTemplar')
            data['player1_Archon'][i] = e.count('Archon')
            data['player1_Immortal'][i] = e.count('Immortal')
            data['player1_Observer'][i] = e.count('Observer')
            data['player1_WarpPrism'][i] = e.count('WarpPrism')
            data['player1_Colossus'][i] = e.count('Colossus')
            data['player1_Disruptor'][i] = e.count('Disruptor')
            data['player1_Phoenix'][i] = e.count('Phoenix')
            data['player1_Carrier'][i] = e.count('Carrier')
            data['player1_VoidRay'][i] = e.count('VoidRay')
            data['player1_Tempest'][i] = e.count('Tempest')
            data['player1_Oracle'][i] = e.count('Oracle')
        elif data['player1_species'][i] == 'Z':
            data['player1_Larva'][i] = e.count('Larva')
            data['player1_Drone'][i] = e.count('Drone')
            data['player1_Overlord'][i] = e.count('Overlord')
            data['player1_Overseer'][i] = e.count('Overseer')
            data['player1_changeling'][i] = e.count('changeling')
            data['player1_Zergling'][i] = e.count('Zergling')
            data['player1_Baneling'][i] = e.count('Baneling')
            data['player1_Queen'][i] = e.count('Queen')
            data['player1_Hydralisk'][i] = e.count('Hydralisk')
            data['player1_Roach'][i] = e.count('Roach')
            data['player1_Mutalisk'][i] = e.count('Mutalisk')
            data['player1_Corruptor'][i] = e.count('Corruptor')
            data['player1_BroodLord'][i] = e.count('BroodLord')
            data['player1_Broodling'][i] = e.count('Broodling')
            data['player1_Infestor'][i] = e.count('Infestor')
            data['player1_InfestedTerran'][i] = e.count('InfestedTerran')
            data['player1_Ultralisk'][i] = e.count('Ultralisk')
    
    return data

In [331]:
player_species = get_units_train(double_train)

  0%|          | 0/38872 [00:00<?, ?it/s]<ipython-input-330-412a3fff0cfa>:110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['player0_SCV'][i] = player0_SCV
<ipython-input-330-412a3fff0cfa>:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['player0_Mule'][i] = player0_Mule
<ipython-input-330-412a3fff0cfa>:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['player0_Marine'][i] = player0_Marine
<ipython-input-330-412a3fff

100%|██████████| 38872/38872 [01:31<00:00, 424.56it/s]


In [332]:
player_species.head(10)

,winner,time,player0_species,player0_event_Ability,player0_event_AddToControlGroup,player0_event_Camera,player0_event_ControlGroup,player0_event_GetControlGroup,player0_event_RightClick,player0_event_Selection,...,player1_Queen,player1_Hydralisk,player1_Roach,player1_Mutalisk,player1_Corruptor,player1_BroodLord,player1_Broodling,player1_Infestor,player1_InfestedTerran,player1_Ultralisk
0,1,7.24,T,"['(1360) - TrainSCV', '(1021) - BuildSupplyDep...","[nan, nan]","['at (145.25, 21.5078125)', 'at (142.99609375,...",[],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","['Location: (136.3759765625, 41.11962890625, 4...","[""['OrbitalCommand [3080001]']"", ""['OrbitalCom...",...,0,0,0,0,0,0,0,0,0,0
1,0,7.28,Z,"['(1820) - MorphDrone', '(4C1) - Patrol; Locat...",[],"['at (24.75, 128.0078125)', 'at (24.75, 125.99...",[],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","['Target: None [00700001]; Location: (19.0, 12...","[""['Hive [3100001]']"", ""['Egg [3140001]', 'Egg...",...,0,0,0,0,0,0,0,0,0,0
2,1,4.27,P,"['(15E0) - TrainProbe', '(15E0) - TrainProbe',...",[nan],"['at (140.0, 162.0078125)', 'at (138.5, 160.75...",[],"[nan, nan, nan]",['Target: DestructibleSignsConstruction [00380...,"[""['Probe [3200001]']"", ""['Probe [3180001]']"",...",...,0,0,0,0,0,0,0,0,0,0
3,0,9.59,T,"['(1360) - TrainSCV', '(1360) - TrainSCV', '(1...","[nan, nan, nan, nan, nan, nan]","['at (145.25, 21.53515625)', 'at (145.25, 22.7...",[],"[nan, nan, nan, nan, nan, nan, nan, nan]","['Location: (135.513671875, 39.0166015625, 491...","[""['SCV [3740001]']"", ""['OrbitalCommand [30800...",...,0,0,0,0,0,0,0,0,0,0
4,1,4.55,Z,"['(1820) - MorphDrone', '(16E0) - BuildHatcher...",[],"['at (127.25, 27.5078125)', 'at (125.9375, 28....",[],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","['Location: (116.93994140625, 46.2587890625, 4...","[""['Hatchery [3380001]']"", ""['Egg [33C0001]', ...",...,1,0,0,0,0,0,0,0,0,0
5,0,10.18,Z,"['(1820) - MorphDrone', '(1822) - MorphOverlor...","[nan, nan]","['at (151.25, 20.5078125)', 'at (151.25, 22.17...",[],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","['Location: (138.77685546875, 26.192138671875,...","[""['Egg [3140001]', 'Egg [3180001]', 'Egg [31C...",...,0,0,0,0,0,0,0,0,0,0
6,1,10.36,Z,"['(1820) - MorphDrone', '(16E2) - BuildExtract...",[],"['at (140.0, 162.0078125)', 'at (138.4375, 160...",[],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","['Location: (109.275634765625, 165.89453125, 4...","[""['Egg [3040001]', 'Egg [3080001]', 'Egg [30C...",...,0,0,0,0,0,0,0,0,0,0
7,1,7.40,P,"['(15E0) - TrainProbe', '(15E0) - TrainProbe',...","[nan, nan, nan, nan, nan]","['at (28.0, 18.5078125)', 'at (29.5, 19.757812...",[],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","['Target: None [00040001]; Location: (29.0, 15...","[""['Nexus [3000001]']"", ""['Probe [32C0001]']"",...",...,0,0,0,0,0,0,0,0,0,0
8,1,6.10,Z,"['(1820) - MorphDrone', '(1822) - MorphOverlor...",[],"['at (22.25, 81.5078125)', 'at (22.0, 81.50781...",[],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","['Location: (47.212158203125, 104.075439453125...","[""['Lair [2DC0001]']"", ""['Egg [2E00001]']"", ""[...",...,0,0,0,0,0,0,0,0,0,0
9,0,5.55,T,"['(1360) - TrainSCV', '(1360) - TrainSCV', '(1...",[nan],"['at (163.25, 18.5078125)', 'at (160.99609375,...",[],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","['Location: (151.775634765625, 29.470703125, 4...","[""['OrbitalCommand [3100001]']"", '[]', ""['SCV ...",...,1,0,1,0,0,0,0,0,0,0


In [333]:
player_species.to_csv('./data/train/player_species.csv',index=False)

In [334]:
def get_apm_train(data):
    col = ['player0_apm',
           'player0_X_apm',
           'player0_cpm',
           'player0_X_cpm',
           'player0_rpm',
           'player0_X_rpm',
           'player0_spm',
           'player0_X_spm',
           'player1_apm',
           'player1_X_apm',
           'player1_cpm',
           'player1_X_cpm',
           'player1_rpm',
           'player1_X_rpm',
           'player1_spm',
           'player1_X_spm']
    frist_train = pd.DataFrame(columns = col)
    
    for i in tqdm(range(len(data))):#len(data)
        player0_action = len(data.loc[i,'player0_event_Ability'])
        player0_X_action = len(data.loc[i,'player0_event_X_Ability'])
        player0_Camera = len(data.loc[i,'player0_event_Camera'])
        player0_X_Camera = len(data.loc[i,'player0_event_X_Camera'])
        player0_RightClick = len(data.loc[i,'player0_event_RightClick'])
        player0_X_RightClick = len(data.loc[i,'player0_event_X_RightClick'] )
        player0_Selection = len(data.loc[i,'player0_event_Selection'])
        player0_X_Selection = len(data.loc[i,'player0_event_X_Selection'])
                                  
        player1_action = len(data.loc[i,'player1_event_Ability'])
        player1_X_action = len(data.loc[i,'player1_event_X_Ability'])
        player1_Camera = len(data.loc[i,'player1_event_Camera'])
        player1_X_Camera = len(data.loc[i,'player1_event_X_Camera'])
        player1_RightClick = len(data.loc[i,'player1_event_RightClick'])
        player1_X_RightClick = len(data.loc[i,'player1_event_X_RightClick'])
        player1_Selection = len(data.loc[i,'player1_event_Selection'])
        player1_X_Selection = len(data.loc[i,'player1_event_X_Selection'])
        
        # 분당 계산
        player0_apm = player0_action/data.loc[i,'time']
        player0_X_apm = player0_X_action/X                          
        player0_cpm = player0_Camera/data.loc[i,'time']
        player0_X_cpm = player0_X_Camera/X                          
        player0_rpm = player0_RightClick/data.loc[i,'time']
        player0_X_rpm = player0_X_RightClick/X                          
        player0_spm = player0_Selection/data.loc[i,'time']
        player0_X_spm = player0_X_Selection/X
                                                         
        player1_apm = player1_action/data.loc[i,'time']
        player1_X_apm = player1_X_action/X                          
        player1_cpm = player1_Camera/data.loc[i,'time']
        player1_X_cpm = player1_X_Camera/X                          
        player1_rpm = player1_RightClick/data.loc[i,'time']
        player1_X_rpm = player1_X_RightClick/X                          
        player1_spm = player1_Selection/data.loc[i,'time']
        player1_X_spm = player1_X_Selection/X        
                                  
        df = pd.Series([player0_apm,
                        player0_X_apm,
                        player0_cpm,
                        player0_X_cpm,
                        player0_rpm,
                        player0_X_rpm,
                        player0_spm,
                        player0_X_spm,
                        player1_apm,
                        player1_X_apm,
                        player1_cpm,
                        player1_X_cpm,
                        player1_rpm,
                        player1_X_rpm,
                        player1_spm,
                        player1_X_spm], index = col)
        frist_train = frist_train.append(df, ignore_index=True)
    return frist_train

In [335]:
apm_train = get_apm_train(player_species)
apm_train = pd.concat([player_species, apm_train], axis =1)

100%|██████████| 38872/38872 [01:19<00:00, 490.01it/s]


In [336]:
apm_train.head(10)

,winner,time,player0_species,player0_event_Ability,player0_event_AddToControlGroup,player0_event_Camera,player0_event_ControlGroup,player0_event_GetControlGroup,player0_event_RightClick,player0_event_Selection,...,player0_spm,player0_X_spm,player1_apm,player1_X_apm,player1_cpm,player1_X_cpm,player1_rpm,player1_X_rpm,player1_spm,player1_X_spm
0,1,7.24,T,"['(1360) - TrainSCV', '(1021) - BuildSupplyDep...","[nan, nan]","['at (145.25, 21.5078125)', 'at (142.99609375,...",[],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","['Location: (136.3759765625, 41.11962890625, 4...","[""['OrbitalCommand [3080001]']"", ""['OrbitalCom...",...,182.044199,106.666667,210.635359,6.666667,1873.756906,90.000000,230.939227,6.666667,332.596685,6.666667
1,0,7.28,Z,"['(1820) - MorphDrone', '(4C1) - Patrol; Locat...",[],"['at (24.75, 128.0078125)', 'at (24.75, 125.99...",[],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","['Target: None [00700001]; Location: (19.0, 12...","[""['Hive [3100001]']"", ""['Egg [3140001]', 'Egg...",...,803.159341,1260.000000,422.252747,260.000000,2169.093407,1726.666667,2459.203297,3796.666667,943.269231,5376.666667
2,1,4.27,P,"['(15E0) - TrainProbe', '(15E0) - TrainProbe',...",[nan],"['at (140.0, 162.0078125)', 'at (138.5, 160.75...",[],"[nan, nan, nan]",['Target: DestructibleSignsConstruction [00380...,"[""['Probe [3200001]']"", ""['Probe [3180001]']"",...",...,217.096019,153.333333,259.484778,333.333333,1798.126464,2473.333333,1461.358314,2476.666667,2204.449649,2946.666667
3,0,9.59,T,"['(1360) - TrainSCV', '(1360) - TrainSCV', '(1...","[nan, nan, nan, nan, nan, nan]","['at (145.25, 21.53515625)', 'at (145.25, 22.7...",[],"[nan, nan, nan, nan, nan, nan, nan, nan]","['Location: (135.513671875, 39.0166015625, 491...","[""['SCV [3740001]']"", ""['OrbitalCommand [30800...",...,301.981230,176.666667,344.525547,336.666667,1726.277372,1373.333333,2514.494265,8586.666667,526.590198,153.333333
4,1,4.55,Z,"['(1820) - MorphDrone', '(16E0) - BuildHatcher...",[],"['at (127.25, 27.5078125)', 'at (125.9375, 28....",[],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","['Location: (116.93994140625, 46.2587890625, 4...","[""['Hatchery [3380001]']"", ""['Egg [33C0001]', ...",...,900.219780,1150.000000,530.549451,163.333333,576.043956,1283.333333,1761.978022,4886.666667,704.175824,1200.000000
5,0,10.18,Z,"['(1820) - MorphDrone', '(1822) - MorphOverlor...","[nan, nan]","['at (151.25, 20.5078125)', 'at (151.25, 22.17...",[],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","['Location: (138.77685546875, 26.192138671875,...","[""['Egg [3140001]', 'Egg [3180001]', 'Egg [31C...",...,1134.086444,1573.333333,468.467583,273.333333,1600.000000,1473.333333,1707.662083,1106.666667,624.165029,330.000000
6,1,10.36,Z,"['(1820) - MorphDrone', '(16E2) - BuildExtract...",[],"['at (140.0, 162.0078125)', 'at (138.4375, 160...",[],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","['Location: (109.275634765625, 165.89453125, 4...","[""['Egg [3040001]', 'Egg [3080001]', 'Egg [30C...",...,784.266409,413.333333,319.208494,420.000000,2755.694981,4076.666667,838.803089,1413.333333,235.135135,250.000000
7,1,7.40,P,"['(15E0) - TrainProbe', '(15E0) - TrainProbe',...","[nan, nan, nan, nan, nan]","['at (28.0, 18.5078125)', 'at (29.5, 19.757812...",[],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","['Target: None [00040001]; Location: (29.0, 15...","[""['Nexus [3000001]']"", ""['Probe [32C0001]']"",...",...,1508.243243,9623.333333,364.594595,346.666667,2128.108108,4343.333333,1133.648649,1943.333333,460.540541,1320.000000
8,1,6.10,Z,"['(1820) - MorphDrone', '(1822) - MorphOverlor...",[],"['at (22.25, 81.5078125)', 'at (22.0, 81.50781...",[],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","['Location: (47.212158203125, 104.075439453125...","[""['Lair [2DC0001]']"", ""['Egg [2E00001]']"", ""[...",...,666.885246,910.000000,301.311475,276.666667,2250.163934,1643.333333,957.213115,1030.000000,164.590164,70.000000
9,0,5.55,T,"['(1360) - TrainSCV', '(1360) 

In [337]:
# 데이터 저장
apm_train.to_csv('./data/train/first_apm_train.csv',index=False)

In [338]:
# 테스트용 ====================================================================================
def toValue (data):
    nRow, nCol = data.shape
    for i in tqdm(range(nRow)):#len(data)
        for j in range(nCol):
            if(data.columns[j] == "winner"):
                continue
            elif(data.columns[j] == "time"):
                continue
            elif(data.columns[j] == "player0_species"):
                if(data.iloc[i,j] == 'P'):
                    data.iloc[i,j] = 0
                elif(data.iloc[i,j] == 'T'):
                    data.iloc[i,j] = 1
                else:
                    data.iloc[i,j] = 2
            elif(data.columns[j] == "player1_species"):
                if(data.iloc[i,j] == 'P'):
                    data.iloc[i,j] = 0
                elif(data.iloc[i,j] == 'T'):
                    data.iloc[i,j] = 1
                else:
                    data.iloc[i,j] = 2
            elif(data.columns[j] == "player0_apm"):
                continue
            elif(data.columns[j] == "player1_apm"):
                continue
            elif(data.columns[j] == "player0_X_apm"):
                continue
            elif(data.columns[j] == "player1_X_apm"):
                continue
            else:
                try:
                    data.iloc[i,j] = len(data.iloc[i,j])
                except Exception as e:
                    continue
    
    x_data = data.iloc[:,1:]
    y_data = data.iloc[:,:1]
    return data, x_data, y_data

In [339]:
_, x_data, y_data = toValue(apm_train)

100%|██████████| 38872/38872 [26:20<00:00, 24.60it/s]


In [340]:
_.head(10)

,winner,time,player0_species,player0_event_Ability,player0_event_AddToControlGroup,player0_event_Camera,player0_event_ControlGroup,player0_event_GetControlGroup,player0_event_RightClick,player0_event_Selection,...,player0_spm,player0_X_spm,player1_apm,player1_X_apm,player1_cpm,player1_X_cpm,player1_rpm,player1_X_rpm,player1_spm,player1_X_spm
0,1,7.24,1,1511,10,14246,2,120,1935,1318,...,182.044199,106.666667,210.635359,6.666667,1873.756906,90.000000,230.939227,6.666667,332.596685,6.666667
1,0,7.28,2,2259,2,9558,2,215,4749,5847,...,803.159341,1260.000000,422.252747,260.000000,2169.093407,1726.666667,2459.203297,3796.666667,943.269231,5376.666667
2,1,4.27,0,714,5,8602,2,15,5055,927,...,217.096019,153.333333,259.484778,333.333333,1798.126464,2473.333333,1461.358314,2476.666667,2204.449649,2946.666667
3,0,9.59,1,1952,30,27224,2,40,8874,2896,...,301.981230,176.666667,344.525547,336.666667,1726.277372,1373.333333,2514.494265,8586.666667,526.590198,153.333333
4,1,4.55,2,1317,2,6642,2,235,3794,4096,...,900.219780,1150.000000,530.549451,163.333333,576.043956,1283.333333,1761.978022,4886.666667,704.175824,1200.000000
5,0,10.18,2,3148,10,10757,2,290,14357,11545,...,1134.086444,1573.333333,468.467583,273.333333,1600.000000,1473.333333,1707.662083,1106.666667,624.165029,330.000000
6,1,10.36,2,2818,2,21287,2,270,13212,8125,...,784.266409,413.333333,319.208494,420.000000,2755.694981,4076.666667,838.803089,1413.333333,235.135135,250.000000
7,1,7.40,0,1988,25,20616,2,2430,8192,11161,...,1508.243243,9623.333333,364.594595,346.666667,2128.108108,4343.333333,1133.648649,1943.333333,460.540541,1320.000000
8,1,6.10,2,1844,2,9611,2,165,4723,4068,...,666.885246,910.000000,301.311475,276.666667,2250.163934,1643.333333,957.213115,1030.000000,164.590164,70.000000
9,0,5.55,1,2200,5,11158,2,270,8288,1726,...,310.990991,1160.000000,315.675676,153.333333,1429.189189,4250.000000,748.828829,783.333333,933.153153,1010.000000


In [341]:
# 데이터 저장
_.to_csv('./data/train/train.csv',index=False)

In [2]:
# 데이터 로드
_ = pd.read_csv('./data/train/train.csv')
x_data = _.iloc[:,1:]
y_data = _.iloc[:,:1]

In [343]:
x_data.head(10)

,time,player0_species,player0_event_Ability,player0_event_AddToControlGroup,player0_event_Camera,player0_event_ControlGroup,player0_event_GetControlGroup,player0_event_RightClick,player0_event_Selection,player0_event_SetControlGroup,...,player0_spm,player0_X_spm,player1_apm,player1_X_apm,player1_cpm,player1_X_cpm,player1_rpm,player1_X_rpm,player1_spm,player1_X_spm
0,7.24,1,1511,10,14246,2,120,1935,1318,15,...,182.044199,106.666667,210.635359,6.666667,1873.756906,90.000000,230.939227,6.666667,332.596685,6.666667
1,7.28,2,2259,2,9558,2,215,4749,5847,85,...,803.159341,1260.000000,422.252747,260.000000,2169.093407,1726.666667,2459.203297,3796.666667,943.269231,5376.666667
2,4.27,0,714,5,8602,2,15,5055,927,10,...,217.096019,153.333333,259.484778,333.333333,1798.126464,2473.333333,1461.358314,2476.666667,2204.449649,2946.666667
3,9.59,1,1952,30,27224,2,40,8874,2896,2,...,301.981230,176.666667,344.525547,336.666667,1726.277372,1373.333333,2514.494265,8586.666667,526.590198,153.333333
4,4.55,2,1317,2,6642,2,235,3794,4096,25,...,900.219780,1150.000000,530.549451,163.333333,576.043956,1283.333333,1761.978022,4886.666667,704.175824,1200.000000
5,10.18,2,3148,10,10757,2,290,14357,11545,15,...,1134.086444,1573.333333,468.467583,273.333333,1600.000000,1473.333333,1707.662083,1106.666667,624.165029,330.000000
6,10.36,2,2818,2,21287,2,270,13212,8125,10,...,784.266409,413.333333,319.208494,420.000000,2755.694981,4076.666667,838.803089,1413.333333,235.135135,250.000000
7,7.40,0,1988,25,20616,2,2430,8192,11161,20,...,1508.243243,9623.333333,364.594595,346.666667,2128.108108,4343.333333,1133.648649,1943.333333,460.540541,1320.000000
8,6.10,2,1844,2,9611,2,165,4723,4068,10,...,666.885246,910.000000,301.311475,276.666667,2250.163934,1643.333333,957.213115,1030.000000,164.590164,70.000000
9,5.55,1,2200,5,11158,2,270,8288,1726,5,...,310.990991,1160.000000,315.675676,153.333333,1429.189189,4250.000000,748.828829,783.333333,933.153153,1010.000000


In [344]:
test = pd.read_csv('./data/test.csv')

In [27]:
test

,game_id,time,player,species,event,event_contents
0,38872,0.00,0,P,Camera,"at (22.25, 81.5078125)"
1,38872,0.00,1,P,Camera,"at (120.25, 153.83984375)"
2,38872,0.01,1,P,Selection,['Nexus [3100001]']
3,38872,0.01,1,P,Ability,(15E0) - TrainProbe
4,38872,0.01,1,P,AddToControlGroup,NaN
...,...,...,...,...,...,...
28714844,55658,4.54,1,T,Right Click,"Location: (120.584228515625, 56.930419921875, ..."
28714845,55658,4.54,0,Z,Camera,"at (70.9921875, 117.65234375)"
28714846,55658,4.54,1,T,Right Click,"Location: (122.146728515625, 55.52099609375, 3..."
28714847,55658,4.54,1,T,Right Click,"Location: (123.6767578125, 54.140625, 33215)"


In [11]:
def get_frist_test(data):
    game_id = data.loc[0,'game_id']
    col = ['time',
           'player0_species',
           'player0_event_Ability',
           'player0_event_AddToControlGroup',
           'player0_event_Camera',
           'player0_event_ControlGroup',
           'player0_event_GetControlGroup',
           'player0_event_RightClick',
           'player0_event_Selection',
           'player0_event_SetControlGroup',
           'player0_event_X_Ability',
           'player0_event_X_AddToControlGroup',
           'player0_event_X_Camera',
           'player0_event_X_ControlGroup',
           'player0_event_X_GetControlGroup',
           'player0_event_X_RightClick',
           'player0_event_X_Selection',
           'player0_event_X_SetControlGroup',
           'player1_species',
           'player1_event_Ability',
           'player1_event_AddToControlGroup',
           'player1_event_Camera',
           'player1_event_ControlGroup',
           'player1_event_GetControlGroup',
           'player1_event_RightClick',
           'player1_event_Selection',
           'player1_event_SetControlGroup',
           'player1_event_X_Ability',
           'player1_event_X_AddToControlGroup',
           'player1_event_X_Camera',
           'player1_event_X_ControlGroup',
           'player1_event_X_GetControlGroup',
           'player1_event_X_RightClick',
           'player1_event_X_Selection',
           'player1_event_X_SetControlGroup']
    frist_train = pd.DataFrame(columns = col)

    time = 0
    player0_species = 'N'
    player0_event_Ability = []
    player0_event_AddToControlGroup = []
    player0_event_Camera = []
    player0_event_ControlGroup = []
    player0_event_GetControlGroup = []
    player0_event_RightClick = []
    player0_event_Selection = []
    player0_event_SetControlGroup = []
    player0_event_X_Ability = []
    player0_event_X_AddToControlGroup = []
    player0_event_X_Camera = []
    player0_event_X_ControlGroup = []
    player0_event_X_GetControlGroup = []
    player0_event_X_RightClick = []
    player0_event_X_Selection = []
    player0_event_X_SetControlGroup = []
    player1_species = 'N'
    player1_event_Ability = []
    player1_event_AddToControlGroup = []
    player1_event_Camera = []
    player1_event_ControlGroup = []
    player1_event_GetControlGroup = []
    player1_event_RightClick = []
    player1_event_Selection = []
    player1_event_SetControlGroup = []
    player1_event_X_Ability = []
    player1_event_X_AddToControlGroup = []
    player1_event_X_Camera = []
    player1_event_X_ControlGroup = []
    player1_event_X_GetControlGroup = []
    player1_event_X_RightClick = []
    player1_event_X_Selection = []
    player1_event_X_SetControlGroup = []
    for i in tqdm(range(len(data))):#len(data)
        if(data.loc[i,'game_id'] == game_id):
            # time
            time = data.loc[i,'time']

            # player_species
            if(data.loc[i,'player'] == 0):
                player0_species = data.loc[i,'species']
            if(data.loc[i,'player'] == 1):
                player1_species = data.loc[i,'species']
            
            if(time <= X):
                # event
                if(data.loc[i,'event'] == "Ability"):
                    if(data.loc[i,'player'] == 0):
                        player0_event_X_Ability.append(data.loc[i,'event_contents'])
                    else:
                        player1_event_X_Ability.append(data.loc[i,'event_contents'])
                if(data.loc[i,'event'] == "AddToControlGroup"):
                    if(data.loc[i,'player'] == 0):
                        player0_event_X_AddToControlGroup.append(data.loc[i,'event_contents'])
                    else:
                        player1_event_X_AddToControlGroup.append(data.loc[i,'event_contents'])
                if(data.loc[i,'event'] == "Camera"):
                    if(data.loc[i,'player'] == 0):
                        player0_event_X_Camera.append(data.loc[i,'event_contents'])
                    else:
                        player1_event_X_Camera.append(data.loc[i,'event_contents'])
                if(data.loc[i,'event'] == "ControlGroup"):
                    if(data.loc[i,'player'] == 0):
                        player0_event_X_ControlGroup.append(data.loc[i,'event_contents'])
                    else:
                        player1_event_X_ControlGroup.append(data.loc[i,'event_contents'])
                if(data.loc[i,'event'] == "GetControlGroup"):
                    if(data.loc[i,'player'] == 0):
                        player0_event_X_GetControlGroup.append(data.loc[i,'event_contents'])
                    else:
                        player1_event_X_GetControlGroup.append(data.loc[i,'event_contents'])
                if(data.loc[i,'event'] == "Right Click"):
                    if(data.loc[i,'player'] == 0):
                        player0_event_X_RightClick.append(data.loc[i,'event_contents'])
                    else:
                        player1_event_X_RightClick.append(data.loc[i,'event_contents'])
                if(data.loc[i,'event'] == "Selection"):
                    if(data.loc[i,'player'] == 0):
                        player0_event_X_Selection.append(data.loc[i,'event_contents'])
                    else:
                        player1_event_X_Selection.append(data.loc[i,'event_contents'])
                if(data.loc[i,'event'] == "SetControlGroup"):
                    if(data.loc[i,'player'] == 0):
                        player0_event_X_SetControlGroup.append(data.loc[i,'event_contents'])
                    else:
                        player1_event_X_SetControlGroup.append(data.loc[i,'event_contents'])

            # event
            if(data.loc[i,'event'] == "Ability"):
                if(data.loc[i,'player'] == 0):
                    player0_event_Ability.append(data.loc[i,'event_contents'])
                else:
                    player1_event_Ability.append(data.loc[i,'event_contents'])
            if(data.loc[i,'event'] == "AddToControlGroup"):
                if(data.loc[i,'player'] == 0):
                    player0_event_AddToControlGroup.append(data.loc[i,'event_contents'])
                else:
                    player1_event_AddToControlGroup.append(data.loc[i,'event_contents'])
            if(data.loc[i,'event'] == "Camera"):
                if(data.loc[i,'player'] == 0):
                    player0_event_Camera.append(data.loc[i,'event_contents'])
                else:
                    player1_event_Camera.append(data.loc[i,'event_contents'])
            if(data.loc[i,'event'] == "ControlGroup"):
                if(data.loc[i,'player'] == 0):
                    player0_event_ControlGroup.append(data.loc[i,'event_contents'])
                else:
                    player1_event_ControlGroup.append(data.loc[i,'event_contents'])
            if(data.loc[i,'event'] == "GetControlGroup"):
                if(data.loc[i,'player'] == 0):
                    player0_event_GetControlGroup.append(data.loc[i,'event_contents'])
                else:
                    player1_event_GetControlGroup.append(data.loc[i,'event_contents'])
            if(data.loc[i,'event'] == "Right Click"):
                if(data.loc[i,'player'] == 0):
                    player0_event_RightClick.append(data.loc[i,'event_contents'])
                else:
                    player1_event_RightClick.append(data.loc[i,'event_contents'])
            if(data.loc[i,'event'] == "Selection"):
                if(data.loc[i,'player'] == 0):
                    player0_event_Selection.append(data.loc[i,'event_contents'])
                else:
                    player1_event_Selection.append(data.loc[i,'event_contents'])
            if(data.loc[i,'event'] == "SetControlGroup"):
                if(data.loc[i,'player'] == 0):
                    player0_event_SetControlGroup.append(data.loc[i,'event_contents'])
                else:
                    player1_event_SetControlGroup.append(data.loc[i,'event_contents'])
        else:
            df = pd.Series([time,
                            player0_species,
                            player0_event_Ability,
                            player0_event_AddToControlGroup,
                            player0_event_Camera,
                            player0_event_ControlGroup,
                            player0_event_GetControlGroup,
                            player0_event_RightClick,
                            player0_event_Selection,
                            player0_event_SetControlGroup,
                            player0_event_X_Ability,
                            player0_event_X_AddToControlGroup,
                            player0_event_X_Camera,
                            player0_event_X_ControlGroup,
                            player0_event_X_GetControlGroup,
                            player0_event_X_RightClick,
                            player0_event_X_Selection,
                            player0_event_X_SetControlGroup,
                            player1_species,
                            player1_event_Ability,
                            player1_event_AddToControlGroup,
                            player1_event_Camera,
                            player1_event_ControlGroup,
                            player1_event_GetControlGroup,
                            player1_event_RightClick,
                            player1_event_Selection,
                            player1_event_SetControlGroup,
                            player1_event_X_Ability,
                            player1_event_X_AddToControlGroup,
                            player1_event_X_Camera,
                            player1_event_X_ControlGroup,
                            player1_event_X_GetControlGroup,
                            player1_event_X_RightClick,
                            player1_event_X_Selection,
                            player1_event_X_SetControlGroup], index = col)
            frist_train = frist_train.append(df, ignore_index=True)

            # 초기화
            time = 0
            player0_species = 'N'
            player0_event_Ability = []
            player0_event_AddToControlGroup = []
            player0_event_Camera = []
            player0_event_ControlGroup = []
            player0_event_GetControlGroup = []
            player0_event_RightClick = []
            player0_event_Selection = []
            player0_event_SetControlGroup = []
            player0_event_X_Ability = []
            player0_event_X_AddToControlGroup = []
            player0_event_X_Camera = []
            player0_event_X_ControlGroup = []
            player0_event_X_GetControlGroup = []
            player0_event_X_RightClick = []
            player0_event_X_Selection = []
            player0_event_X_SetControlGroup = []
            player1_species = 'N'
            player1_event_Ability = []
            player1_event_AddToControlGroup = []
            player1_event_Camera = []
            player1_event_ControlGroup = []
            player1_event_GetControlGroup = []
            player1_event_RightClick = []
            player1_event_Selection = []
            player1_event_SetControlGroup = []
            player1_event_X_Ability = []
            player1_event_X_AddToControlGroup = []
            player1_event_X_Camera = []
            player1_event_X_ControlGroup = []
            player1_event_X_GetControlGroup = []
            player1_event_X_RightClick = []
            player1_event_X_Selection = []
            player1_event_X_SetControlGroup = []
            
            game_id = data.loc[i,'game_id']

            # time
            time = data.loc[i,'time']

            # player_species
            if(data.loc[i,'player'] == 0):
                player0_species = data.loc[i,'species']
            if(data.loc[i,'player'] == 1):
                player1_species = data.loc[i,'species']
            
            if(time <= X):
                # event
                if(data.loc[i,'event'] == "Ability"):
                    if(data.loc[i,'player'] == 0):
                        player0_event_X_Ability.append(data.loc[i,'event_contents'])
                    else:
                        player1_event_X_Ability.append(data.loc[i,'event_contents'])
                if(data.loc[i,'event'] == "AddToControlGroup"):
                    if(data.loc[i,'player'] == 0):
                        player0_event_X_AddToControlGroup.append(data.loc[i,'event_contents'])
                    else:
                        player1_event_X_AddToControlGroup.append(data.loc[i,'event_contents'])
                if(data.loc[i,'event'] == "Camera"):
                    if(data.loc[i,'player'] == 0):
                        player0_event_X_Camera.append(data.loc[i,'event_contents'])
                    else:
                        player1_event_X_Camera.append(data.loc[i,'event_contents'])
                if(data.loc[i,'event'] == "ControlGroup"):
                    if(data.loc[i,'player'] == 0):
                        player0_event_X_ControlGroup.append(data.loc[i,'event_contents'])
                    else:
                        player1_event_X_ControlGroup.append(data.loc[i,'event_contents'])
                if(data.loc[i,'event'] == "GetControlGroup"):
                    if(data.loc[i,'player'] == 0):
                        player0_event_X_GetControlGroup.append(data.loc[i,'event_contents'])
                    else:
                        player1_event_X_GetControlGroup.append(data.loc[i,'event_contents'])
                if(data.loc[i,'event'] == "Right Click"):
                    if(data.loc[i,'player'] == 0):
                        player0_event_X_RightClick.append(data.loc[i,'event_contents'])
                    else:
                        player1_event_X_RightClick.append(data.loc[i,'event_contents'])
                if(data.loc[i,'event'] == "Selection"):
                    if(data.loc[i,'player'] == 0):
                        player0_event_X_Selection.append(data.loc[i,'event_contents'])
                    else:
                        player1_event_X_Selection.append(data.loc[i,'event_contents'])
                if(data.loc[i,'event'] == "SetControlGroup"):
                    if(data.loc[i,'player'] == 0):
                        player0_event_X_SetControlGroup.append(data.loc[i,'event_contents'])
                    else:
                        player1_event_X_SetControlGroup.append(data.loc[i,'event_contents'])

            # event
            if(data.loc[i,'event'] == "Ability"):
                if(data.loc[i,'player'] == 0):
                    player0_event_Ability.append(data.loc[i,'event_contents'])
                else:
                    player1_event_Ability.append(data.loc[i,'event_contents'])
            if(data.loc[i,'event'] == "AddToControlGroup"):
                if(data.loc[i,'player'] == 0):
                    player0_event_AddToControlGroup.append(data.loc[i,'event_contents'])
                else:
                    player1_event_AddToControlGroup.append(data.loc[i,'event_contents'])
            if(data.loc[i,'event'] == "Camera"):
                if(data.loc[i,'player'] == 0):
                    player0_event_Camera.append(data.loc[i,'event_contents'])
                else:
                    player1_event_Camera.append(data.loc[i,'event_contents'])
            if(data.loc[i,'event'] == "ControlGroup"):
                if(data.loc[i,'player'] == 0):
                    player0_event_ControlGroup.append(data.loc[i,'event_contents'])
                else:
                    player1_event_ControlGroup.append(data.loc[i,'event_contents'])
            if(data.loc[i,'event'] == "GetControlGroup"):
                if(data.loc[i,'player'] == 0):
                    player0_event_GetControlGroup.append(data.loc[i,'event_contents'])
                else:
                    player1_event_GetControlGroup.append(data.loc[i,'event_contents'])
            if(data.loc[i,'event'] == "Right Click"):
                if(data.loc[i,'player'] == 0):
                    player0_event_RightClick.append(data.loc[i,'event_contents'])
                else:
                    player1_event_RightClick.append(data.loc[i,'event_contents'])
            if(data.loc[i,'event'] == "Selection"):
                if(data.loc[i,'player'] == 0):
                    player0_event_Selection.append(data.loc[i,'event_contents'])
                else:
                    player1_event_Selection.append(data.loc[i,'event_contents'])
            if(data.loc[i,'event'] == "SetControlGroup"):
                if(data.loc[i,'player'] == 0):
                    player0_event_SetControlGroup.append(data.loc[i,'event_contents'])
                else:
                    player1_event_SetControlGroup.append(data.loc[i,'event_contents'])

    df = pd.Series([time,
                    player0_species,
                    player0_event_Ability,
                    player0_event_AddToControlGroup,
                    player0_event_Camera,
                    player0_event_ControlGroup,
                    player0_event_GetControlGroup,
                    player0_event_RightClick,
                    player0_event_Selection,
                    player0_event_SetControlGroup,
                    player0_event_X_Ability,
                    player0_event_X_AddToControlGroup,
                    player0_event_X_Camera,
                    player0_event_X_ControlGroup,
                    player0_event_X_GetControlGroup,
                    player0_event_X_RightClick,
                    player0_event_X_Selection,
                    player0_event_X_SetControlGroup,
                    player1_species,
                    player1_event_Ability,
                    player1_event_AddToControlGroup,
                    player1_event_Camera,
                    player1_event_ControlGroup,
                    player1_event_GetControlGroup,
                    player1_event_RightClick,
                    player1_event_Selection,
                    player1_event_SetControlGroup,
                    player1_event_X_Ability,
                    player1_event_X_AddToControlGroup,
                    player1_event_X_Camera,
                    player1_event_X_ControlGroup,
                    player1_event_X_GetControlGroup,
                    player1_event_X_RightClick,
                    player1_event_X_Selection,
                    player1_event_X_SetControlGroup], index = col)
    frist_train = frist_train.append(df, ignore_index=True)
    return frist_train

frist_test = get_frist_test(test)

In [4]:
frist_test.to_csv('./data/train/first_X_test.csv',index=False)

NameError: name 'frist_test' is not defined

In [5]:
frist_test = pd.read_csv('./data/train/first_X_test.csv')

In [6]:
frist_test.head(10)

,time,player0_species,player0_event_Ability,player0_event_AddToControlGroup,player0_event_Camera,player0_event_ControlGroup,player0_event_GetControlGroup,player0_event_RightClick,player0_event_Selection,player0_event_SetControlGroup,...,player1_event_Selection,player1_event_SetControlGroup,player1_event_X_Ability,player1_event_X_AddToControlGroup,player1_event_X_Camera,player1_event_X_ControlGroup,player1_event_X_GetControlGroup,player1_event_X_RightClick,player1_event_X_Selection,player1_event_X_SetControlGroup
0,7.06,P,"['(15E0) - TrainProbe', '(15E0) - TrainProbe',...",[nan],"['at (22.25, 81.5078125)', 'at (24.9453125, 78...",[],"[nan, nan, nan, nan, nan, nan, nan, nan, nan]","['Target: None [01FC0001]; Location: (18.0, 87...","[""['Probe [2F80001]']"", ""['Probe [2F40001]']"",...","[nan, nan, nan, nan, nan]",...,"[""['Nexus [3100001]']"", ""['Probe [32C0001]']"",...",[],"['(15E0) - TrainProbe', '(1541) - BuildPylon; ...","[nan, nan]","['at (120.25, 153.83984375)', 'at (120.25, 153...",[],"[nan, nan]","['Location: (122.58837890625, 148.31689453125,...","[""['Nexus [3100001]']"", ""['Probe [32C0001]']""]",[]
1,6.07,Z,"['(1822) - MorphOverlord', '(1820) - MorphDron...",[],"['at (24.75, 128.0078125)', 'at (156.0, 40.484...","[nan, nan, nan, nan]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","['Location: (142.35107421875, 48.42236328125, ...","[""['Egg [3140001]', 'Egg [3180001]', 'Egg [31C...",[],...,"[""['Probe [3780001]']"", ""['Probe [3780001]']"",...","[nan, nan, nan, nan, nan]","['(15E0) - TrainProbe', '(1541) - BuildPylon; ...",[],"['at (151.25, 20.5078125)', 'at (151.25, 20.07...",[],[nan],"['Location: (144.859130859375, 46.604736328125...",[],"[nan, nan]"
2,4.41,T,"['(1360) - TrainSCV', '(1021) - BuildSupplyDep...",[],"['at (151.25, 20.5078125)', 'at (151.25, 21.63...",[],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","['Location: (146.873779296875, 30.279296875, 4...","[""['OrbitalCommand [3100001]']"", ""['SCV [31400...","[nan, nan, nan, nan, nan, nan, nan]",...,"[""['OrbitalCommand [3440001]']"", ""['SCV [37800...","[nan, nan, nan, nan, nan]","['(1360) - TrainSCV', '(1021) - BuildSupplyDep...",[],"['at (24.75, 128.0078125)', 'at (24.75, 127.34...",[],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","['Location: (29.10205078125, 118.415771484375,...","[""['OrbitalCommand [3440001]']"", ""['SCV [37800...","[nan, nan]"
3,7.58,T,"['(1360) - TrainSCV', '(1360) - TrainSCV', '(F...","[nan, nan, nan, nan, nan, nan, nan, nan]","['at (152.25, 142.0078125)', 'at (152.25, 140....",[],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",['Target: DestructibleSignsConstruction [018C0...,"[""['OrbitalCommand [3100001]']"", ""['SCV [33000...","[nan, nan, nan, nan, nan, nan, nan, nan, nan]",...,"[""['Nexus [3440001]']"", ""['Probe [3580001]', '...","[nan, nan]","['(15E0) - TrainProbe', '(15E0) - TrainProbe',...",[],"['at (31.75, 22.5078125)', 'at (31.75, 26.4531...",[],[],"['Location: (26.953857421875, 26.69775390625, ...","[""['Nexus [3440001]']"", ""['Probe [3580001]', '...",[]
4,2.50,Z,"['(1820) - MorphDrone', '(1820) - MorphDrone',...","[nan, nan]","['at (135.25, 31.5078125)', 'at (135.25, 32.19...",[],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","['Location: (34.505615234375, 71.298095703125,...","[""['Hatchery [2DC0001]']"", ""['Egg [2E00001]', ...","[nan, nan]",...,"[""['Egg [3240001]', 'Egg [3280001]', 'Egg [32C...",[nan],['(16E2) - BuildExtractor; Target: Destructibl...,[],"['at (22.25, 81.5078125)']",[],[],"['Location: (25.53466796875, 77.536376953125, ...","[""['Egg [3240001]', 'Egg [3280001]', 'Egg [32C...",[nan]
5,2.32,T,"['(1360) - TrainSCV', '(1022) - BuildRefinery;...",[],"['at (163.25, 18.5078125)', 'at (160.421875, 1...",[],"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","['Location: (168.3359375, 27.87158203125, 4910...","[""['SCV [32C0001]']"", ""['SCV [3380001]']"", ""['...","[nan, nan, nan]",...,"[""['SCV [3600001]']"", ""['SCV [3480001]']"", ""['...","[nan, nan, nan, nan]","['(1021) - 

In [12]:
units_test = get_units_train(frist_test)

  0%|          | 0/16787 [00:00<?, ?it/s]<ipython-input-8-412a3fff0cfa>:133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['player0_Probe'][i] = e.count('Probe')
<ipython-input-8-412a3fff0cfa>:134: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['player0_MothershipCore'][i] = e.count('MothershipCore')
<ipython-input-8-412a3fff0cfa>:135: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['player0_Mothership'][i] = e.count('Mothe

100%|██████████| 16787/16787 [00:32<00:00, 510.58it/s]


In [13]:
units_test.loc[0,'player0_Probe']

12

In [16]:
amp_test = get_apm_train(units_test)
amp_test = pd.concat([units_test, amp_test], axis =1)

100%|██████████| 16787/16787 [00:26<00:00, 622.09it/s]


In [17]:
value_test,_,_ = toValue(amp_test)

100%|██████████| 16787/16787 [06:05<00:00, 45.98it/s]


In [18]:
value_test.head(10)

,time,player0_species,player0_event_Ability,player0_event_AddToControlGroup,player0_event_Camera,player0_event_ControlGroup,player0_event_GetControlGroup,player0_event_RightClick,player0_event_Selection,player0_event_SetControlGroup,...,player0_spm,player0_X_spm,player1_apm,player1_X_apm,player1_cpm,player1_X_cpm,player1_rpm,player1_X_rpm,player1_spm,player1_X_spm
0,7.06,0,1499,5,7079,2,45,14053,1597,25,...,226.203966,76.666667,249.150142,263.333333,2071.388102,4586.666667,965.864023,546.666667,190.368272,153.333333
1,6.07,2,1402,2,6211,20,90,2792,2827,2,...,465.733114,826.666667,171.334432,260.000000,743.163097,1956.666667,668.863262,466.666667,81.713344,6.666667
2,4.41,1,1718,2,7186,2,315,4418,1711,35,...,387.981859,906.666667,334.693878,273.333333,1505.442177,1900.000000,1407.256236,356.666667,201.360544,176.666667
3,7.58,1,4215,40,16044,2,1275,13476,5821,45,...,767.941953,820.000000,372.295515,333.333333,1412.796834,606.666667,2530.079156,6156.666667,612.005277,650.000000
4,2.50,2,322,10,4036,2,1935,1669,760,10,...,304.000000,963.333333,157.600000,336.666667,2016.800000,86.666667,170.400000,536.666667,416.000000,920.000000
5,2.32,1,332,2,1666,2,2315,847,4119,15,...,1775.431034,140.000000,171.120690,203.333333,1554.310345,3096.666667,1075.000000,1593.333333,140.086207,140.000000
6,3.30,0,416,2,2424,2,5,548,374,5,...,113.333333,153.333333,130.000000,240.000000,1406.060606,1906.666667,803.939394,1820.000000,2023.333333,670.000000
7,9.59,1,5961,30,12432,2,960,10169,4664,30,...,486.339937,176.666667,300.000000,76.666667,1319.082377,1476.666667,1093.952033,1546.666667,3869.968717,10063.333333
8,8.52,1,5544,2,5796,2,940,8827,21678,45,...,2544.366197,2413.333333,530.516432,163.333333,2734.389671,1973.333333,1137.323944,1466.666667,1520.774648,2246.666667
9,10.10,1,3795,20,22282,2,115,8904,12566,2,...,1244.158416,5163.333333,263.366337,163.333333,1823.168317,4480.000000,1846.039604,2086.666667,3088.811881,4100.000000


In [3]:
# 데이터 저장
value_test.to_csv('./data/train/test.csv',index=False)

NameError: name 'value_test' is not defined

In [4]:
# 데이터 로드
value_test = pd.read_csv('./data/train/test.csv')

In [6]:
y_data1

0        1
1        0
2        1
3        0
4        1
        ..
38867    0
38868    0
38869    0
38870    1
38871    1
Name: winner, Length: 38872, dtype: int64

In [48]:
# 데이터 로드
_ = pd.read_csv('./data/train/train.csv')
# 데이터 로드
value_test = pd.read_csv('./data/train/test.csv')
mapdata = pd.read_csv('./data/train/mapdata.csv')

In [34]:
mapdata_copy = mapdata.copy()
mapdata_copy = pd.concat([mapdata,mapdata_copy])
mapdata_copy.to_csv('./data/train/mapdata_copy.csv', index = False)

In [35]:
mapdata_copy = pd.read_csv('./data/train/mapdata_copy.csv')

In [36]:
_ = pd.concat([_,mapdata_copy], axis = 1)


In [37]:
_ = _.drop(['player0_start', 'player1_start'], axis=1)


In [49]:
x_data = _.iloc[:,1:]
y_data = _.iloc[:,:1]
y_data1 = y_data.T.squeeze()

In [44]:
_

,winner,time,player0_species,player0_event_Ability,player0_event_AddToControlGroup,player0_event_Camera,player0_event_ControlGroup,player0_event_GetControlGroup,player0_event_RightClick,player0_event_Selection,...,player0_X_spm,player1_apm,player1_X_apm,player1_cpm,player1_X_cpm,player1_rpm,player1_X_rpm,player1_spm,player1_X_spm,map
0,1,7.24,1,1511,10,14246,2,120,1935,1318,...,106.666667,210.635359,6.666667,1873.756906,90.000000,230.939227,6.666667,332.596685,6.666667,3.0
1,0,7.28,2,2259,2,9558,2,215,4749,5847,...,1260.000000,422.252747,260.000000,2169.093407,1726.666667,2459.203297,3796.666667,943.269231,5376.666667,6.0
2,1,4.27,0,714,5,8602,2,15,5055,927,...,153.333333,259.484778,333.333333,1798.126464,2473.333333,1461.358314,2476.666667,2204.449649,2946.666667,4.0
3,0,9.59,1,1952,30,27224,2,40,8874,2896,...,176.666667,344.525547,336.666667,1726.277372,1373.333333,2514.494265,8586.666667,526.590198,153.333333,3.0
4,1,4.55,2,1317,2,6642,2,235,3794,4096,...,1150.000000,530.549451,163.333333,576.043956,1283.333333,1761.978022,4886.666667,704.175824,1200.000000,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38867,0,0.39,2,86,2,1492,2,2,194,155,...,263.333333,317.948718,153.333333,6656.410256,8570.000000,141.025641,183.333333,58.974359,76.666667,4.0
38868,0,0.37,1,176,2,851,2,120,987,522,...,1543.333333,113.513514,70.000000,1843.243243,2190.000000,1767.567568,2180.000000,702.702703,866.666667,4.0
38869,0,8.22,0,1395,2,9902,2,2,2556,1744,...,230.000000,266.909976,70.000000,2397.201946,2696.666667,648.296837,370.000000,200.364964,176.666667,3.0
38870,1,9.59,0,1227,2,20536,2,40,3341,1608,...,153.333333,352.450469,70.000000,1775.912409,9826.666667,715.954119,173.333333,169.134515,106.666667,4.0


In [40]:
value_test

,time,player0_species,player0_event_Ability,player0_event_AddToControlGroup,player0_event_Camera,player0_event_ControlGroup,player0_event_GetControlGroup,player0_event_RightClick,player0_event_Selection,player0_event_SetControlGroup,...,player0_spm,player0_X_spm,player1_apm,player1_X_apm,player1_cpm,player1_X_cpm,player1_rpm,player1_X_rpm,player1_spm,player1_X_spm
0,7.06,0,1499,5,7079,2,45,14053,1597,25,...,226.203966,76.666667,249.150142,263.333333,2071.388102,4586.666667,965.864023,546.666667,190.368272,153.333333
1,6.07,2,1402,2,6211,20,90,2792,2827,2,...,465.733114,826.666667,171.334432,260.000000,743.163097,1956.666667,668.863262,466.666667,81.713344,6.666667
2,4.41,1,1718,2,7186,2,315,4418,1711,35,...,387.981859,906.666667,334.693878,273.333333,1505.442177,1900.000000,1407.256236,356.666667,201.360544,176.666667
3,7.58,1,4215,40,16044,2,1275,13476,5821,45,...,767.941953,820.000000,372.295515,333.333333,1412.796834,606.666667,2530.079156,6156.666667,612.005277,650.000000
4,2.50,2,322,10,4036,2,1935,1669,760,10,...,304.000000,963.333333,157.600000,336.666667,2016.800000,86.666667,170.400000,536.666667,416.000000,920.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16782,6.14,2,1727,10,7748,2,3465,7603,10042,30,...,1635.504886,6533.333333,383.550489,140.000000,1617.426710,2176.666667,1423.289902,1643.333333,613.843648,1943.333333
16783,9.58,2,3106,2,26470,2,1085,8880,13023,35,...,1359.394572,1883.333333,419.832985,273.333333,2093.841336,3310.000000,1057.828810,710.000000,403.235908,196.666667
16784,6.24,0,515,2,10367,2,2,6427,2954,5,...,473.397436,153.333333,297.435897,260.000000,1675.000000,2263.333333,2341.666667,3590.000000,405.448718,2626.666667
16785,1.17,1,230,2,527,2,2,579,125,10,...,106.837607,173.333333,226.495726,206.666667,1694.017094,1453.333333,751.282051,2203.333333,2891.452991,2756.666667


In [50]:
# CatBoost 모델링
def catboost_modeling(x_train, y_train, x_test, grow_policy, depth, learning_rate, l2_leaf_reg, random_seed, n):
  
    # 빈 Series인 test_pred 생성
    test_pred = pd.Series([0 for x in range(len(x_test))], index=x_test.index)
    
    # 10-fold 모델링을 n회 반복할 것
    for i in range(n):
        kf = KFold(n_splits=10, random_state=random_seed+i)
        for train_index, valid_index in kf.split(x_train):
            train_X, train_y = x_train.iloc[train_index], y_train[train_index]
            valid_X, valid_y = x_train.iloc[valid_index], y_train[valid_index]
      
            # catBoost(grow_policy='Depthwise')
            model = CatBoostClassifier(eval_metric = 'AUC',              # AUC로 성능 측정
                                 iterations = 25000,               # 반복횟수 최대 25000
                                 metric_period = 25000,            # 중간결과 출력X
                                 early_stopping_rounds = 1000,     # 1000iteration 동안 AUC 증가 없으면 학습 중단
                                 task_type = 'GPU',                # GPU 사용
                                 grow_policy = grow_policy,        # 트리 노드 생성 방식
                                                                   # 1) Depthwise(지정한 depth에 이를 때까지 level 순으로 노드 분할)
                                                                   # 2) Lossguide(loss 변화가 큰 순으로 노드 분할)
                                 depth = depth,                    # 트리 깊이
                                 learning_rate = learning_rate,    # 러닝레이트
                                 l2_leaf_reg = l2_leaf_reg,        # L2 정규화
                                 random_seed = random_seed+i,      # 랜덤시드 고정
                                 )
            # 모델 학습
            model.fit(train_X, train_y, eval_set=(valid_X, valid_y))

            test_pred += model.predict_proba(x_test)[:,1] /(10*n)
            
    return test_pred

In [51]:
data1 = catboost_modeling(x_data, y_data1, value_test, 'Depthwise', 10, 0.02423, 20.35, 2014, 2)

C:\Users\User\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


0:	learn: 0.7324644	test: 0.5850938	best: 0.5850938 (0)	total: 63.6ms	remaining: 26m 29s
bestTest = 0.6631620526
bestIteration = 529
Shrink model to first 530 iterations.
0:	learn: 0.7355630	test: 0.5926233	best: 0.5926233 (0)	total: 51.6ms	remaining: 21m 30s
bestTest = 0.6639384031
bestIteration = 610
Shrink model to first 611 iterations.
0:	learn: 0.7353105	test: 0.5921175	best: 0.5921175 (0)	total: 52.3ms	remaining: 21m 47s
bestTest = 0.6796646714
bestIteration = 580
Shrink model to first 581 iterations.
0:	learn: 0.7346236	test: 0.5745933	best: 0.5745933 (0)	total: 53.1ms	remaining: 22m 6s
bestTest = 0.6677948236
bestIteration = 760
Shrink model to first 761 iterations.
0:	learn: 0.7331026	test: 0.5905364	best: 0.5905364 (0)	total: 49.5ms	remaining: 20m 38s
bestTest = 0.6789987087
bestIteration = 1245
Shrink model to first 1246 iterations.
0:	learn: 0.7407395	test: 0.5804911	best: 0.5804911 (0)	total: 51.6ms	remaining: 21m 30s
bestTest = 0.6645966172
bestIteration = 874
Shrink mode

In [ ]:
data2 = catboost_modeling(x_data, y_data1, value_test, 'Lossguide', 8, 0.01063, 5.127, 2014, 2)

0:	learn: 0.6201973	test: 0.6010407	best: 0.6010407 (0)	total: 30.4ms	remaining: 12m 40s


In [ ]:
data3 = catboost_modeling(x_data, y_data1, value_test, 'Depthwise', 12, 0.01564, 49.99, 2022, 2)

In [ ]:
data4 = catboost_modeling(x_data, y_data1, value_test, 'Lossguide', 16, 0.01213, 5.027, 2022, 2)

In [52]:
# 최종 모델 앙상블
sample_submission = pd.read_csv('./data/sample_submission.csv')
data_final = pd.DataFrame(data1) # (data1+data2)/2 *1/3 + (data3+data4)/2 *2/3
sample_submission.iloc[:,1:] = data_final[0]
sample_submission.to_csv('./result/data_final.csv', index =False)
sample_submission

,game_id,winner
0,38872,0.535709
1,38873,0.537647
2,38874,0.368869
3,38875,0.196825
4,38876,0.374228
...,...,...
16782,55654,0.798821
16783,55655,0.452945
16784,55656,0.736216
16785,55657,0.512121
